# Object Detection

Project Path

In [1]:
path = "C:/Users/Pongphan/Desktop/"

Load Model Parameter (.h5 file)

In [2]:
from keras.models import load_model

model = load_model(path + "best_model_fold_MobileNet3.h5")
#model.summary()

Custom Function for Import test Images

In [3]:
import cv2
import numpy as np

def ObjectDet(num):
    img = cv2.imread(path + f"evegg ({num}).tif")
    ratio = 1

    img = cv2.resize(img, (img.shape[1]//ratio,img.shape[0]//ratio), interpolation = cv2.INTER_AREA)

    box_size_y = 370
    box_size_x = 370
    
    step_size = 50

    img_output = np.array(img)
    img_cont = np.zeros((img_output.shape[0],img_output.shape[1]))
    result = 0

    for i in range(0,img_output.shape[0]-box_size_y,step_size):
        for j in range(0,img_output.shape[1]-box_size_x,step_size):
            img_patch = img_output[i:i+box_size_y,j:j+box_size_x]
            img_patch = cv2.resize(img_patch,(128,128),interpolation=cv2.INTER_AREA)
            img_patch = [img_patch]
            img_patch = np.array(img_patch)
            
            y_outp = model.predict(img_patch,verbose=0)

            if result < y_outp[0][1] and y_outp[0][1] > 0.8:
                result = y_outp[0][1]
                img_cont[i+(box_size_y//2),j+(box_size_x//2)] = y_outp[0][1]*255

    boxlocat = []
    
    if result != 0:
        label = "Ov:"+format(result, f".{2}f")
        boxlocat = boxlocation(img_cont,box_size_x//2)
        img_output = drawbox(img,label,boxlocat[0],boxlocat[1],boxlocat[2],boxlocat[3],box_size_x//2)

    image_path = path + f"heatmap_{num}.png"
    cv2.imwrite(image_path, img_cont)

    image_path = path + f"output_{num}"+str(boxlocat)+".png"
    cv2.imwrite(image_path, img_output)

Custom Function for Identified Bounding Box

In [4]:
def boxlocation(img_c,box_size):
    a = b = c = d = 0
    for i in range(img_c.shape[0]):
        for j in range(img_c.shape[1]):
            if a==0 and img_c[i,j]>0:
                a = i
            if a!=0 and img_c[i,j]>0:
                b = i
    for j in range(img_c.shape[1]):
        for i in range(img_c.shape[0]):
            if c==0 and img_c[i,j]>0:
                c = j
            if c!=0 and img_c[i,j]>0:
                d = j
    locat = [a-box_size,b+box_size,c-box_size,d+box_size]
    return locat

Custom Function for Draw Bounding Box

In [5]:
def drawbox(img,label,a,b,c,d,box_size):
    image = cv2.rectangle(img, (c,a), (d,b), (0, 255, 0), 2)
    image = cv2.putText(image, label, (c+box_size,a-10), cv2.FONT_HERSHEY_TRIPLEX, 2, (255, 0, 255), 1)
    return image

Detect Ov egg in Selected Image

In [6]:
import datetime

current_time = datetime.datetime.now().time()
print("time:", current_time)
ObjectDet('3000')
current_time = datetime.datetime.now().time()
print("time:", current_time)

print("Detection Complete !")

time: 17:19:59.688176
time: 17:21:13.421887
Detection Complete !
